# Settings the units for a simulation

For convenience, REBOUND offers simple functionality for converting units.  One implicitly sets the units for the simulation through the values used for the initial conditions, but one has to set the appropriate value for the gravitational constant G, and sometimes it is convenient to get the output in different units.

The default value for G is 1, so one can:

a) use units for the initial conditions where G=1 (e.g., AU, $M_\odot$, yr/$2\pi$)

b) set G manually to the value appropriate for the adopted initial conditions, e.g., to use SI units,

In [1]:
import rebound
import math
rebound.G = 6.674e-11

c) set rebound.units:

In [2]:
rebound.units = ('yr', 'AU', 'Msun')
print("G = {0}.  Should be {1}.".format(rebound.G, 4*math.pi**2))

G = 39.476926421373.  Should be 39.47841760435743.


When you set the units, REBOUND converts G to the appropriate value for the units passed (must pass exactly 3 units, but can be in any order).  Note that if you are interested in high precision, you have to be quite particular about the exact units.  The reason for the above discrepancy is mostly that we follow the convention of defining a year as 365.25 days (a Julian year), whereas the Earth's sidereal orbital period is closer to 365.256 days (and at even finer level, Venus and Mercury modify the orbital period).

##Adding particles

If you use rebound.units at all, you need to set the units before adding any particles.  You can then add particles in any of the ways described in [WHFast.ipynb](WHFast.ipynb).  You can also add particles drawing from the horizons database (see [Churyumov-Gerasimenko.ipynb](Churyumov-Gerasimenko.ipynb)).  If you don't set the units ahead of time, HORIZONS will return initial conditiosn in units of AU, $M_\odot$ and yrs/$2\pi$, such that G=1.  

Above we switched to units of AU, $M_\odot$ and yrs, so when we add Earth:

In [3]:
rebound.add('Earth')
ps = rebound.particles
import math
print("v = {0}".format(math.sqrt(ps[0].vx**2 + ps[0].vy**2 + ps[0].vz**2)))

Searching NASA Horizons for 'Earth'... Found: Earth-Moon Barycenter (3).
v = 6.1895535126146735


we see that the velocity is correctly set to approximately $2\pi$ AU/yr.

If you'd like to enter the initial conditions in one set of units, and then use a different set for the simulation, you can use the rebound.convert_particle_units function, which converts both the initial conditions and G.  Since we added Earth above, we have to reset rebound; otherwise we'll get an error saying that we can't set the units with particles already loaded:

In [4]:
rebound.reset()
rebound.units = ('m', 's', 'kg')
rebound.add(m=1.99e30)
rebound.add(m=5.97e24,a=1.5e11)

rebound.convert_particle_units('AU', 'yr', 'Msun')
rebound.status()

---------------------------------
Rebound version:     	0.4.2
Build on:            	b'Jun  2 2015 17:06:20'
Number of particles: 	2
Simulation time:     	0.000000
---------------------------------
<rebound.Particle object, m=1.0007547141626585 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, m=3.0022641424879755e-06 x=1.0026880683402668 y=0.0 z=0.0 vx=0.0 vy=6.2770157204114065 vz=0.0>
---------------------------------


We first set the units to SI, added (approximate values for) the Sun and Earth in these units, and switched to AU, yr, $M_\odot$.  You can see that the particle states were converted correctly--the Sun has a mass of about 1, and the Earth has a distance of about 1.

Note that when you pass orbital elements to rebound.add, you *must* make sure G is set correctly ahead of time (through either 3 of the methods above), since it will use the value of rebound.G to generate the velocities:

In [5]:
rebound.reset()
print("G = {0}".format(rebound.G))
rebound.add(m=1.99e30)
rebound.add(m=5.97e24,a=1.5e11)
rebound.status()

G = 1.0
---------------------------------
Rebound version:     	0.4.2
Build on:            	b'Jun  2 2015 17:06:20'
Number of particles: 	2
Simulation time:     	0.000000
---------------------------------
<rebound.Particle object, m=1.99e+30 x=0.0 y=0.0 z=0.0 vx=0.0 vy=0.0 vz=0.0>
<rebound.Particle object, m=5.97e+24 x=150000000000.0 y=0.0 z=0.0 vx=0.0 vy=3642349031.4173164 vz=0.0>
---------------------------------


The orbital speed of Earth is $\sim 3\times 10^5$ m/s, but since we didn't correctly set G, we get $\sim 3\times 10^9$ m/s, so the Earth would fly off the Sun in this simulation.